# Algoritmos Genéticos

Los Algoritmos Genéticos (GA) son métodos adaptativos que pueden usarse para resolver problemas de búsqueda y  optimización. Están basados en el proceso genético de los organismos vivos. A
lo largo de las generaciones, las poblaciones evolucionan en la naturaleza de acorde con los principios de la selección natural y la supervivencia de los más aptos, postulados por Darwin (1859), matemáticamente explicados por Ronald Fisher (1930) y generalizados por George R. Price (1972).
Por imitación de este proceso, los Algoritmos Genéticos son capaces de ir creando soluciones para
problemas del mundo real. La evolución de dichas soluciones hacia valores óptimos del problema
depende en buena medida de una adecuada codificación de las mismas.

Los principios básicos de los Algoritmos Genéticos fueron establecidos por Holland (1975), y se
encuentran bien descritos en varios textos – Goldberg (1989), Davis (1991), Michalewicz (1992),
Reeves (1993).

## Pasos del algoritmo

* Determinar la población
* Calcular la aptitud de dicha población
* Selección de los mejores individuos para generar la siguiente generación
* Apareamiento
* Mutación
* Selección de la descendencia


En primer lugar vamos a generar unas funciones para seguir estos pasos:

In [1]:
# librerias

import numpy as np

In [2]:
# aptitud:
# se calcula el valor de la aptitud en la población 
# esta función se calcula como la suma de los productos entre los inputs y sus correspondientes pesos

def aptitud(X, w):
    
    apt = np.sum(X*w, axis=1)
    
    return apt

In [3]:
# seleccion de los mejores individuos en la presente generación como progenitores de la siguiente generación

def posibles_padres(poblacion, aptitud, num_padres):
    
    padres = np.empty((num_padres, poblacion.shape[1]))
    
    for i in range(num_padres):
        max_aptitud_id = np.where(aptitud == np.max(aptitud))
        max_aptitud_id = max_aptitud_id[0][0]
        padres[i, :] = poblacion[max_aptitud_id, :]
        aptitud[max_aptitud_id] = -99999999999          # a minimos para no elegir al mismo
        
    return padres

In [4]:
# cruce

def cruce(padres, num_hijos):
    hijos = np.empty(num_hijos)
    # punto de cruce entre dos progenitores. Usualmente es el centro (la mitad de los genes)
    punto_cruce = np.uint8(num_hijos[1]/2)

    for k in range(num_hijos[0]):
        # indice del primer progenitor 
        padre1_idx = k%padres.shape[0]
        # indice del segundo progenitor
        padre2_idx = (k+1)%padres.shape[0]
        
        # la nueva descendencia tendra la primera mitad de sus genes del primer progenitor
        hijos[k, 0:punto_cruce] = padres[padre1_idx, 0:punto_cruce]
        # la nueva descendencia tendra la segunda mitad de sus genes del segundo progenitor
        hijos[k, punto_cruce:] = padres[padre2_idx, punto_cruce:]
        
    return hijos


Existen diferentes tipos de cruce: uniforme, un punto, dos puntos, etc...En este caso se usa cruce en un punto

In [5]:
# mutación, cambia un solo gen en cada hijo aleatoriamente 
# se usa la representación decimal de los genes, pueden ser enteros, binarios, etc...

def mutacion(mut_hijos):
    
    for i in range(mut_hijos.shape[0]):
        random = np.random.uniform(-1.0, 1.0, 1)
        mut_hijos[i, 4] = mut_hijos[i, 4] + random
        
    return mut_hijos


En este caso se ha optado por una mutación aleatoria, pero la mutación puede seguir cualquier distribución:
uniforme, normal, etc...

## Ejemplo de aplicación

En éste ejemplo vamos a maximizar una función lineal, representada por la siguiente ecuación:

**Y = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + w6x6**

La ecuación tiene 6 entradas (x1-x6) y 6 pesos (w1-w6).

Para unos valores dados de x, por ejemplo (x1,x2,x3,x4,x5,x6)=(4,-2,7,5,11,1), tratamos de encontrar aquellos pesos que maximicen el valor de Y. 

La idea de maximizar ésta ecuación es simple. Para x positivas se buscan valores de w positivos lo más grandes posibles, y para x negativos valores de w lo más negativos posibles.Pero vamos a hacerlo implementando un algoritmo genético. 

En principio, se crea la lista de X y se determina el número de parámetros w a optimizar:


In [6]:
# valores fijos de X
X = [4,-2,3.5,5,-11,-4.7]

# numero de pesos que vamos a optimizar
num_pesos = 6

El siguiente paso es definir la población inicial. Basándose en el número de pesos, cada cromosoma in la población tendrá 6 genes, un gen por cada peso.Pero, ¿cuántas soluciones hay para la población?. No hay un valor fijo, se puede escoger un número cualquiera, asi que se pondrá como una variable más en el código para poder cambiarla. Lo siguiente sería definir el número total de individuos en la población:


In [7]:
sol_pob = 8     # soluciones posibles en la poblacion

# se define el tamaño de la poblacion 
pob_size = (sol_pob, num_pesos) 

# creando la poblacion inicial
nueva_pob = np.random.uniform(low=-4.0, high=4.0, size=pob_size)

Se inicia aleatoriamente la población inicial. De acuerdo con los parámetros escogidos, las dimensiones son (8, 6).
Esto significa 8 cromosomas con 6 genes cada uno, uno para cada peso. Se muestra la población:

In [8]:
nueva_pob

array([[-0.72189015,  2.8894041 , -2.29901094,  3.33220696,  3.46060772,
         3.07128832],
       [-0.90199941, -0.78178018,  3.22220007,  1.11600828, -1.81247402,
        -3.59996858],
       [-2.06207578, -3.68310994, -1.59853487, -2.54856734, -2.68924485,
        -0.04568113],
       [ 0.68571943,  1.50846357,  1.54450052, -3.7218215 , -0.39543571,
        -2.29610381],
       [ 1.68619447, -2.94126315, -1.63754933,  3.41091674, -2.95262997,
        -3.06407845],
       [ 0.20180404,  0.43353111, -1.49612024,  0.07196183,  1.52513159,
         2.22578542],
       [-1.47427824,  2.51577733, -1.94800919, -2.52792993, -3.28231177,
         0.63278833],
       [-1.50428152, -0.95091628, -2.73269284,  3.82299807,  1.92065479,
        -3.24243953]])

# Algoritmo Genético

Ahora se aplican las funciones creadas, con la mutación y el cruce entre la población, e iterando un número dado de generaciones

In [9]:
num_posibles_padres = 4    # numero de posibles padres en la poblacion
num_gen = 10               # numero de generaciones

In [10]:
for gen in range(num_gen):
    print ('Generacion : ', gen)
    # se mide la aptitud de cada cromosoma en la poblacion
    apt = aptitud(X, nueva_pob)

    # se seleccionan los mejores progenitores para el cruce
    padres = posibles_padres(nueva_pob, apt, num_posibles_padres)

    # se genera la siguiente generacion
    descendencia = cruce(padres, num_hijos=(pob_size[0]-padres.shape[0], num_pesos))

    # se añaden las mutaciones
    descendencia_mutada = mutacion(descendencia)

    # se crea una nueva poblacion basada en los padres y la descendencia
    nueva_pob[0:padres.shape[0], :] = padres
    nueva_pob[padres.shape[0]:, :] = descendencia_mutada

    # mejor resultado
    print ('Mejor resultado : ', np.max(np.sum(nueva_pob*X, axis=1)))
    print ()
    
# mejor resultado despues de todas la generaciones
# primero se calcula la aptitud para cada solucion en al generacion final 
apt = aptitud(X, nueva_pob)

# luego se devuelve el indice de esa solucion que corresponde al resultadon con mejor aptitud 
mejores_id = np.where(apt == np.max(apt))

print ('Mejor solucion' , nueva_pob[mejores_id, :])
print ()
print ('Mejor solucion de aptitud' , apt[mejores_id])

Generacion :  0
Mejor resultado :  70.83056365111689

Generacion :  1
Mejor resultado :  87.11637173029433

Generacion :  2
Mejor resultado :  94.16949611235327

Generacion :  3
Mejor resultado :  106.89700547016784

Generacion :  4
Mejor resultado :  111.99460120900225

Generacion :  5
Mejor resultado :  115.14434262180679

Generacion :  6
Mejor resultado :  115.14434262180679

Generacion :  7
Mejor resultado :  123.63502408659306

Generacion :  8
Mejor resultado :  125.4442878320192

Generacion :  9
Mejor resultado :  134.2168102153403

Mejor solucion [[[-0.90199941 -0.78178018  3.22220007  3.41091674 -8.5025268
   -3.06407845]]]

Mejor solucion de aptitud [134.21681022]
